## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

#### План работ:
1. Общий взгляд на данные.
2. Предобработка данных.
3. Обработка пропусков.
4. Замена типа данных.
5. Обработка дубликатов.
5. Приведение к общему виду данных(лемматизация).
6. Категоризация данных.
7. Ответы на поставленные вопросы.

#### Описание данных:
- children — количество детей в семье
- days_employed — общий трудовой стаж в днях
- dob_years — возраст клиента в годах
- education — уровень образования клиента
- education_id — идентификатор уровня образования
- family_status — семейное положение
- family_status_id — идентификатор семейного положения
- gender — пол клиента
- income_type — тип занятости
- debt — имел ли задолженность по возврату кредитов
- total_income — ежемесячный доход
- purpose — цель получения кредита

#### Поставленные вопросы:
- Есть ли зависимость между наличием детей и возвратом кредита в срок?
- Есть ли зависимость между семейным положением и возвратом кредита в срок?
- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
- Как разные цели кредита влияют на его возврат в срок?

### Шаг 1. Откройте файл с данными и изучите общую информацию. 

## Изучение данных

*Изучим каждый столбец на корректность значений. После изучения, составим дальнейший план по исправлению данных

In [1]:
import pandas as pd
from  IPython.display import display   #импортируем библиотеку для вывод таблиц наглядно

In [2]:
def df_info(df):
    df.info()
    display(df.head(10))
    print(f'Кол-во дубликатов: {df.duplicated().sum()}')
    print()
    print(f'Кол-во пропусков: {df.isnull().sum()}')

In [ ]:
df = pd.read_csv('/datasets/data.csv')

In [4]:
df_info(df)

Кол-во дубликатов: 54

Кол-во пропусков: children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64


*В датафрейме присутствуют 2174 строк с пропусками.*
*Причины могут быть разные:*
1. *Технические(при переносе данных из одной среды в другую)*
2. *Ошибка заполняющих*
3. *Ошибка людей, которые эти данные предоставляли*

*Также нужно проверить, пропуски в столбцах `days_employed` и `total_income` находятся под одинаковыми строками или нет*

*Следует привести некоторые признаки к правильному типу данных*

In [5]:
miss = pd.isnull(df)
omissions_true = miss[(miss['days_employed'] == True) & (miss['total_income'] == True)].count()
print(omissions_true)

children            2174
days_employed       2174
dob_years           2174
education           2174
education_id        2174
family_status       2174
family_status_id    2174
gender              2174
income_type         2174
debt                2174
total_income        2174
purpose             2174
dtype: int64


In [6]:
miss = pd.isnull(df)
omissions_true = miss[(miss['days_employed'] == True) & (miss['total_income'] == True)].count()
print(omissions_true)

children            2174
days_employed       2174
dob_years           2174
education           2174
education_id        2174
family_status       2174
family_status_id    2174
gender              2174
income_type         2174
debt                2174
total_income        2174
purpose             2174
dtype: int64


*Пропуски в столбцах совпадают*

In [7]:
print(df['children'].value_counts())

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64


*В глаза попадают такие значения, как "-1" и "20". Кол-во детей "-1" явно не может быть, "20" детей тоже как-то многовато, скорее всего число детей было записано как 2.0*

In [8]:
print(df['days_employed'].value_counts())

-986.927316     1
-7026.359174    1
-4236.274243    1
-6620.396473    1
-1238.560080    1
               ..
-2849.351119    1
-5619.328204    1
-448.829898     1
-1687.038672    1
-582.538413     1
Name: days_employed, Length: 19351, dtype: int64


*Присутствуют отрицательные и пропущенные значения (причина возникновения может быть свяазана с переносом данных из одной системы в другую, либо ошибка заполняющего, также есть данные, которые превышают возраст самих клиентов (аномалия). Опишем метод исправления во 2 шаге*

In [9]:
print(df['dob_years'].value_counts())

35    617
40    609
41    607
34    603
38    598
42    597
33    581
39    573
31    560
36    555
44    547
29    545
30    540
48    538
37    537
50    514
43    513
32    510
49    508
28    503
45    497
27    493
56    487
52    484
47    480
54    479
46    475
58    461
57    460
53    459
51    448
59    444
55    443
26    408
60    377
25    357
61    355
62    352
63    269
64    265
24    264
23    254
65    194
66    183
22    183
67    167
21    111
0     101
68     99
69     85
70     65
71     58
20     51
72     33
19     14
73      8
74      6
75      1
Name: dob_years, dtype: int64


*Присутствуют 0 значения. Во 2 шаге опишем метод исправления*

In [10]:
print(df['education'].value_counts())

среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
Ученая степень             1
УЧЕНАЯ СТЕПЕНЬ             1
Name: education, dtype: int64


*Следует переписать слова прописью (str.lower())*

In [11]:
print(df['education_id'].value_counts())

1    15233
0     5260
2      744
3      282
4        6
Name: education_id, dtype: int64


*Проблем не выявлено*

In [12]:
 print(df['family_status_id'].value_counts())

0    12380
1     4177
4     2813
3     1195
2      960
Name: family_status_id, dtype: int64


*Проблем не выявлено*

In [13]:
print(df['gender'].value_counts())

F      14236
M       7288
XNA        1
Name: gender, dtype: int64


*пол 'XNA' следует удалить*

In [14]:
print(df['income_type'].value_counts())

сотрудник          11119
компаньон           5085
пенсионер           3856
госслужащий         1459
безработный            2
предприниматель        2
студент                1
в декрете              1
Name: income_type, dtype: int64


*Проблем не выявлено*

In [15]:
print(df['debt'].value_counts())

0    19784
1     1741
Name: debt, dtype: int64


*Проблем не выявлено*

In [16]:
print(df['total_income'].value_counts())

112874.418757    1
133912.272223    1
182036.676828    1
122421.963500    1
198271.837248    1
                ..
133299.194693    1
115080.782380    1
84896.781597     1
153838.839212    1
150014.128510    1
Name: total_income, Length: 19351, dtype: int64


*Проблем пока не выявлено, кроме пропусков. Во 2 шаге опишем метод исправления*

In [17]:
print(df['purpose'].value_counts())

свадьба                                   797
на проведение свадьбы                     777
сыграть свадьбу                           774
операции с недвижимостью                  676
покупка коммерческой недвижимости         664
операции с жильем                         653
покупка жилья для сдачи                   653
операции с коммерческой недвижимостью     651
покупка жилья                             647
жилье                                     647
покупка жилья для семьи                   641
строительство собственной недвижимости    635
недвижимость                              634
операции со своей недвижимостью           630
строительство жилой недвижимости          626
покупка недвижимости                      624
строительство недвижимости                620
покупка своего жилья                      620
ремонт жилью                              612
покупка жилой недвижимости                607
на покупку своего автомобиля              505
заняться высшим образованием      

*Причины дублируются под разными словосочетаниями, нужно будет привести к общему через лемматизацию*

In [18]:
print(df['income_type'][df['total_income'].isnull()].value_counts())

сотрудник          1105
компаньон           508
пенсионер           413
госслужащий         147
предприниматель       1
Name: income_type, dtype: int64


*Пропусчены значения по данным:*
*1. у сотрудников (1105 строк)*
*2. у компаньонов (508)*
*3. у пенсионеров (413)*
*4. у госслужающих (147)*
*5. у предпринимателей (1)*

In [19]:
print(df['family_status'].value_counts())

женат / замужем          12380
гражданский брак          4177
Не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64


*Следует сократить вариацию на "В браке" и "Не в браке"*

### Вывод

Данные были изучены. Можно приступать к работе по их обработке

### Шаг 2. Предобработка данных

### Кол-во детей у клиентов

In [20]:
# Исправляем отрицательные значения
df['children'] = df['children'].abs()

# Изменим значение 20 на 2
df['children'] = df['children'].replace(20, 2)

# Проверим исправления
print(df['children'].value_counts())

0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64


### Уменьшим регистр в столбце "Образование"

In [21]:
df['education'] = df['education'].str.lower()

print(df['education'].value_counts())

среднее                15233
высшее                  5260
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64


### Обработка пропусков

## Заполнение пропущенных значений

1. Пропущенные значения в столбце "дохода" следует заполнить относительно "Типа занятности"
2. Нулевые значения в столбце "Возраст клиента" следует заполнить относительно "Типа занятости"
3. Пропущенные значения в столбце "Стаж" следует заполнить относительно "Возраста клиента"

### Заполнение пропущенных значений в столбце "Доход"

In [22]:
df_groupby_income_type_mean = df.groupby('income_type')['total_income'].mean()      #Найдем средний доход по типу занятости
df_groupby_income_type_median = df.groupby('income_type')['total_income'].median()   #Найдем медиану по типу занятости

print('Среднее', df_groupby_income_type_mean)
print()
print('Медиана', df_groupby_income_type_median)

Среднее income_type
безработный        131339.751676
в декрете           53829.130729
госслужащий        170898.309923
компаньон          202417.461462
пенсионер          137127.465690
предприниматель    499163.144947
сотрудник          161380.260488
студент             98201.625314
Name: total_income, dtype: float64

Медиана income_type
безработный        131339.751676
в декрете           53829.130729
госслужащий        150447.935283
компаньон          172357.950966
пенсионер          118514.486412
предприниматель    499163.144947
сотрудник          142594.396847
студент             98201.625314
Name: total_income, dtype: float64


*Среднее и медиана показывают приблизительно одинаковые значения.*
*Отличия есть у госслужающих, компаньонов, пенсионеров и сотрудников.*
*Так как данные сравниваются между людьми одной группы/должности, то следует брать среднее значение и вставлять их в пропусщенные ячейки*

In [23]:
df['total_income'] = df['total_income'].fillna(df.groupby('income_type')['total_income'].transform('mean'))

*Получилось заполнить пропуски в столбце "Доход". Пропуски заполнялись сред. величинами в зависимости от "типа занятости"*

### Изменение 0-ых значений в столбце "Возраст"

In [24]:
df_groupby_income_type_mean = df.groupby('income_type')['dob_years'].mean()  #Найдем средний возраст клиентов по типу занятости
print('Среднее', df_groupby_income_type_mean)
print()
df_groupby_income_type_median = df.groupby('income_type')['dob_years'].median() #найдем медиану по типу занятости
print('Медиана', df_groupby_income_type_median)

Среднее income_type
безработный        38.000000
в декрете          39.000000
госслужащий        40.636737
компаньон          39.697542
пенсионер          59.063019
предприниматель    42.500000
сотрудник          39.821027
студент            22.000000
Name: dob_years, dtype: float64

Медиана income_type
безработный        38.0
в декрете          39.0
госслужащий        40.0
компаньон          39.0
пенсионер          60.0
предприниматель    42.5
сотрудник          39.0
студент            22.0
Name: dob_years, dtype: float64


In [25]:
df['dob_years'] = df['dob_years'].fillna(df.groupby('income_type')['dob_years'].transform('median'))

*Нулевые значения были заменены. Но при этом анамальные значения все же присутствуют в столбце "Стаж". df имеет данные по столбцу "Стаж", которые многократно превышают реальный возраст клиента*

### Изменим вариацию семейного положения 

- `в браке` – женат / замужем, гражданский брак
- `не в браке` – не женат / не замужем, в разводе, вдовец / вдова

In [26]:
def categorize_family_status(value):
    if value == 'женат / замужем' or value == 'гражданский брак':
        return 'в браке'
    
    return 'не в браке'

In [27]:
df['family_status'] = df['family_status'].apply(categorize_family_status)

### Заполнение пропущенных значений в столбце "Стаж"

In [28]:
# Исправим отрицательные значения
df['days_employed'] = df['days_employed'].abs()

df_groupby_dob_years_mean = df.groupby('dob_years')['days_employed'].mean()      #Найдем средний стаж по возрасту
df_groupby_dob_years_median = df.groupby('dob_years')['days_employed'].median()   #Найдем медиану по возрасту

print('Среднее', df_groupby_dob_years_mean)
print()
print('Медиана', df_groupby_dob_years_median)

Среднее dob_years
0      69516.105102
19       633.678086
20       684.944308
21       709.440930
22      2793.321854
23       827.309437
24      1026.405485
25      1088.406453
26      3214.617245
27      3725.820728
28      2180.056457
29      1553.823200
30      1696.039355
31      2331.246721
32      3877.197466
33      3241.413096
34      4141.242883
35      2755.226484
36      4496.534938
37      5879.263440
38      7641.485465
39      5277.456460
40      7076.521918
41      6400.926359
42      8885.465110
43      9418.428468
44      9952.060042
45     11748.039481
46     14121.191502
47     14223.685900
48     18149.683912
49     26679.790846
50     50493.031018
51     68425.772762
52     82803.402429
53     94073.249426
54    126757.781831
55    152264.946612
56    157417.505013
57    193689.068297
58    188571.489818
59    227496.974028
60    262925.277586
61    246731.048849
62    274381.283863
63    293383.721011
64    287670.475737
65    287448.538552
66    313613.718234
67

*Ужасные и аномальные значения мы имеем в столбце "Стаж". многие настолько колоссальные, что превышают возраст клиента в многократно. Будем работать снова со средними величинами, так как они +/- близки к правде.*

In [29]:
df['days_employed'] = df['days_employed'].fillna(df.groupby('dob_years')['days_employed'].transform('mean'))

*Пропуски в столбце "Стаж" заполнены.*

### Вывод

*Столбец "Доход" нам в дальнейшем понадобиться, чтобы ответить на вопросы в 3 шаге, а вот пропуски в столбце "Стаж" и 0-ые значения в столбце "Возраст" нам в дальнейшем не нужны.*

### Замена типа данных

In [30]:
df_edited = df.copy()
df_edited['days_employed'] = df_edited['days_employed'].astype('int')
df_edited['total_income'] = df_edited['total_income'].astype('int')
df_edited.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


### Вывод

*Для изменения типа данных из вещественных в целочисленные воспользуемся методом astype(). to_numeric() не подходит, так как нет смысла из вещественных в вещественные исправлять*

### Обработка дубликатов

In [31]:
print('До удаления дубликатов')
print(df_edited.duplicated().sum())
print()
df_edited = df_edited.drop_duplicates().reset_index(drop = True)
print('После удаления дубликатов')
print(df_edited.duplicated().sum())
print()
print('Кол-во строк после удаления дубликатов: {}'.format(len(df_edited)))

До удаления дубликатов
71

После удаления дубликатов
0

Кол-во строк после удаления дубликатов: 21454


### Лемматизация

В столбце purpose обнаружены слова, записанные в разных формах. Необходимо провести лемматизацию. Выделяю все леммы в столбце purpose:

In [32]:
list_purpose = df_edited['purpose'].unique()
print(list_purpose)

['покупка жилья' 'приобретение автомобиля' 'дополнительное образование'
 'сыграть свадьбу' 'операции с жильем' 'образование'
 'на проведение свадьбы' 'покупка жилья для семьи' 'покупка недвижимости'
 'покупка коммерческой недвижимости' 'покупка жилой недвижимости'
 'строительство собственной недвижимости' 'недвижимость'
 'строительство недвижимости' 'на покупку подержанного автомобиля'
 'на покупку своего автомобиля' 'операции с коммерческой недвижимостью'
 'строительство жилой недвижимости' 'жилье'
 'операции со своей недвижимостью' 'автомобили' 'заняться образованием'
 'сделка с подержанным автомобилем' 'получение образования' 'автомобиль'
 'свадьба' 'получение дополнительного образования' 'покупка своего жилья'
 'операции с недвижимостью' 'получение высшего образования'
 'свой автомобиль' 'сделка с автомобилем' 'профильное образование'
 'высшее образование' 'покупка жилья для сдачи' 'на покупку автомобиля'
 'ремонт жилью' 'заняться высшим образованием']


In [33]:
from pymystem3 import Mystem
m = Mystem()
str_purpose = (', ').join(list_purpose)
lemmas = m.lemmatize(str_purpose)

str_purpose_done = ('').join(lemmas)
print(str_purpose_done)

покупка жилье, приобретение автомобиль, дополнительный образование, сыграть свадьба, операция с жилье, образование, на проведение свадьба, покупка жилье для семья, покупка недвижимость, покупка коммерческий недвижимость, покупка жилой недвижимость, строительство собственный недвижимость, недвижимость, строительство недвижимость, на покупка подержать автомобиль, на покупка свой автомобиль, операция с коммерческий недвижимость, строительство жилой недвижимость, жилье, операция со свой недвижимость, автомобиль, заниматься образование, сделка с подержать автомобиль, получение образование, автомобиль, свадьба, получение дополнительный образование, покупка свой жилье, операция с недвижимость, получение высокий образование, свой автомобиль, сделка с автомобиль, профильный образование, высокий образование, покупка жилье для сдача, на покупка автомобиль, ремонт жилье, заниматься высокий образование



Из полученной строки с целями кредита, обоначим основные цели кредита:
- недвижимость, жилье, жилой.
- автомобиль.
- образование.
- свадьба.
- коммерческий.
- ремонт.

In [34]:
# Функция для распределения целей кредита:
def lemmatization_purpose(list):
    for purpose in list:
        if 'автомобиль' in purpose:
            print(purpose + ': автокредит')
        elif 'образование' in purpose:
            print(purpose + ': образование')
        elif 'свадьба' in purpose:
            print(purpose + ': свадьба')
        elif 'ремонт' in purpose:
            print(purpose + ': ремонт жилья')
        elif 'коммерческий' in purpose:
            print(purpose + ': коммерческая недвижимость')
        elif 'недвижимость' in purpose or 'жилье' in purpose or 'жилой' in purpose:
            print(purpose + ': жилая недвижимость')

In [35]:
list_purpose_done = str_purpose_done.split(', ')
lemmatization_purpose(list_purpose_done)

покупка жилье: жилая недвижимость
приобретение автомобиль: автокредит
дополнительный образование: образование
сыграть свадьба: свадьба
операция с жилье: жилая недвижимость
образование: образование
на проведение свадьба: свадьба
покупка жилье для семья: жилая недвижимость
покупка недвижимость: жилая недвижимость
покупка коммерческий недвижимость: коммерческая недвижимость
покупка жилой недвижимость: жилая недвижимость
строительство собственный недвижимость: жилая недвижимость
недвижимость: жилая недвижимость
строительство недвижимость: жилая недвижимость
на покупка подержать автомобиль: автокредит
на покупка свой автомобиль: автокредит
операция с коммерческий недвижимость: коммерческая недвижимость
строительство жилой недвижимость: жилая недвижимость
жилье: жилая недвижимость
операция со свой недвижимость: жилая недвижимость
автомобиль: автокредит
заниматься образование: образование
сделка с подержать автомобиль: автокредит
получение образование: образование
автомобиль: автокредит
свадь

*Функция работает на лемматизированном списке целей. Применим функцию на таблице с данными*

In [36]:
def lemmatization_purpose(df):
    purpose = df['purpose']
    lemmas = m.lemmatize(purpose)
    for purpose in lemmas:
        if 'автомобиль' in purpose:
            return 'автокредит'
        elif 'образование' in purpose:
            return 'образование'
        elif 'свадьба' in purpose:
            return 'свадьба'
        elif 'ремонт' in purpose:
            return 'ремонт жилья'
        elif 'коммерческий' in purpose:
            return 'коммерческая недвижимость'
        elif 'недвижимость' in purpose or 'жилье' in purpose or 'жилой' in purpose:
            return 'жилая недвижимость'

Добавим новый столбец lemm_purpose в таблицу, применив метод apply. Первый аргумент метода - функция распределения целей кредита, фторой аргумент axis - нужен для того, чтобы в функцию поступали данные в строковой форме

In [37]:
df_edited['purpose_after_lemmatization'] = df_edited.apply(lemmatization_purpose, axis=1)
display(df_edited.head())

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_after_lemmatization
0,1,8437,42,высшее,0,в браке,0,F,сотрудник,0,253875,покупка жилья,жилая недвижимость
1,1,4024,36,среднее,1,в браке,0,F,сотрудник,0,112080,приобретение автомобиля,автокредит
2,0,5623,33,среднее,1,в браке,0,M,сотрудник,0,145885,покупка жилья,жилая недвижимость
3,3,4124,32,среднее,1,в браке,0,M,сотрудник,0,267628,дополнительное образование,образование
4,0,340266,53,среднее,1,в браке,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба


### Вывод

*С помощью лемматизации находим основу каждого слова и добавляем его в новый столбец "purpose_after_lemmatization"*
*Причиной использования лемматизации вместо стемминга является то, что лемматизация учитывает контекст и преобразует слово в его значимую базовую форму, тогда как стемминг просто удаляет последние несколько символов, что часто приводит к неверному значению и орфографическим ошибкам.*

### Категоризация данных

### Категоризация по целям кредита

In [38]:
print(df_edited['purpose_after_lemmatization'].value_counts())

жилая недвижимость           8893
автокредит                   4306
образование                  4013
свадьба                      2324
коммерческая недвижимость    1311
ремонт жилья                  607
Name: purpose_after_lemmatization, dtype: int64


### Категоризация по доходу

In [39]:
#С помощью метода qcut() выделил 5 основных групп по доходу. Данный метод хорош тем, что мы для каждой группы выделили ровное кол-во строк

df_edited['income_level'] = pd.qcut(df_edited['total_income'], 5)
print(df_edited['income_level'].value_counts())
print()

df_edited['income_level_2'] = pd.qcut(df_edited['total_income'], 20) #Для проверки увеличим число категорий
print(df_edited['income_level_2'].value_counts())

(214618.2, 2265604.0]    4291
(162638.8, 214618.2]     4291
(98537.6, 135448.4]      4291
(20666.999, 98537.6]     4291
(135448.4, 162638.8]     4290
Name: income_level, dtype: int64

(151887.0, 161380.0]     2004
(187304.4, 202417.0]     1527
(331569.0, 2265604.0]    1073
(64570.65, 78721.1]      1073
(88734.35, 98537.6]      1073
(98537.6, 107623.0]      1073
(116008.6, 126296.55]    1073
(126296.55, 135448.4]    1073
(269825.5, 331569.0]     1073
(141725.0, 151887.0]     1073
(162638.8, 172464.9]     1073
(172464.9, 187304.4]     1073
(214618.2, 237540.55]    1073
(20666.999, 64570.65]    1073
(135448.4, 141725.0]     1072
(107623.0, 116008.6]     1072
(78721.1, 88734.35]      1072
(237540.55, 269825.5]    1072
(202417.0, 214618.2]      618
(161380.0, 162638.8]      141
Name: income_level_2, dtype: int64


### Вывод

*Сделав категоризацию по доходу, можно будет ответь на вопрос "Есть ли зависимость между уровнем дохода и возвратом кредита в срок?"*

### Шаг 3. Ответьте на вопросы

- #### Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [40]:
df_edited_children = df_edited.copy()

#Формируем сводную таблицу
children_pivot = df_edited_children.pivot_table(index=['children'], values = 'debt', aggfunc = ['count', 'sum'], margins = True)
children_pivot.columns = ['Общее кол-во клиентов', 'Кол-во клиентов с долгами по кредиту'] #Переименовываем столбцы
children_pivot['ratio'] = children_pivot['Кол-во клиентов с долгами по кредиту'] / children_pivot['Общее кол-во клиентов']

#Находим % соотношения
children_pivot['ratio'] = children_pivot['ratio'].apply('{:.2%}'.format)
children_pivot

,Общее кол-во клиентов,Кол-во клиентов с долгами по кредиту,ratio
children,,,
0,14091,1063,7.54%
1,4855,445,9.17%
2,2128,202,9.49%
3,330,27,8.18%
4,41,4,9.76%
5,9,0,0.00%
All,21454,1741,8.12%


### Вывод

*Соотношения не сильно варьируются между собой, чтобы сделать вывод, что присутствует зависимость. Можно заметить, что разница между отсутствием детей и их наличием имеется, но она не так велика, чтобы сделать заключение касательно наличия зависимости детей и возврата кредита в срок. Поэтому зависимости нет. Да и присутствуют клиенты, у которых 5 детей, но нет долгов по погашению.*

- #### Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [41]:
#Проводим все то, что делали выше с детьми
family_pivot = df_edited.pivot_table(index=['family_status'], values = 'debt', aggfunc = ['count', 'sum'], margins = True)
family_pivot.columns = ['Общее кол-во клиентов', 'Кол-во клиентов с долгами по кредиту']
family_pivot['ratio'] = family_pivot['Кол-во клиентов с долгами по кредиту'] / family_pivot['Общее кол-во клиентов']
family_pivot['ratio'] = family_pivot['ratio'].apply('{:.2%}'.format)
family_pivot

,Общее кол-во клиентов,Кол-во клиентов с долгами по кредиту,ratio
family_status,,,
в браке,16490,1319,8.00%
не в браке,4964,422,8.50%
All,21454,1741,8.12%


### Вывод

*Соотношения между клиентами с долгами по кредиту и с клиентами без долгов выдают малые значения, для того, чтобы подвердить закономерность семейного положения и возврата кредита в срок.Поэтому закономерности нет*

- #### Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [42]:
#Категоризацию по уровню дохода мы уже провели во 2 шаге

#Проводим все то, что делали выше с детьми

income_pivot = df_edited.pivot_table(index = 'income_level', values = 'debt', aggfunc = ['count', 'sum'], margins = True)
income_pivot.columns = ['Общее кол-во клиентов', 'Кол-во клиентов с долгами по кредиту']
income_pivot['ratio'] = income_pivot['Кол-во клиентов с долгами по кредиту'] / income_pivot['Общее кол-во клиентов']
income_pivot['ratio'] = income_pivot['ratio'].apply('{:.2%}'.format)
display(income_pivot)
print()

#Построим 2 св. таблицу для проверки и анализа в сравнение с 1
income_pivot2 = df_edited.pivot_table(index = 'income_level_2', values = 'debt', aggfunc = ['count', 'sum'], margins = True)
income_pivot2.columns = ['Общее кол-во клиентов', 'Кол-во клиентов с долгами по кредиту']
income_pivot2['ratio'] = income_pivot2['Кол-во клиентов с долгами по кредиту'] / income_pivot2['Общее кол-во клиентов']
income_pivot2['ratio'] = income_pivot2['ratio'].apply('{:.2%}'.format)
display(income_pivot2)

,Общее кол-во клиентов,Кол-во клиентов с долгами по кредиту,ratio
income_level,,,
"(20666.999, 98537.6]",4291,344,8.02%
"(98537.6, 135448.4]",4291,355,8.27%
"(135448.4, 162638.8]",4290,384,8.95%
"(162638.8, 214618.2]",4291,358,8.34%
"(214618.2, 2265604.0]",4291,300,6.99%
All,21454,1741,8.12%


,Общее кол-во клиентов,Кол-во клиентов с долгами по кредиту,ratio
income_level_2,,,
"(20666.999, 64570.65]",1073,71,6.62%
"(64570.65, 78721.1]",1073,86,8.01%
"(78721.1, 88734.35]",1072,90,8.40%
"(88734.35, 98537.6]",1073,97,9.04%
"(98537.6, 107623.0]",1073,83,7.74%
"(107623.0, 116008.6]",1072,96,8.96%
"(116008.6, 126296.55]",1073,98,9.13%
"(126296.55, 135448.4]",1073,78,7.27%
"(135448.4, 141725.0]",1072,100,9.33%


### Вывод

*Общая картина показывает, что зависимость между уровнем дохода и возвратом кредита скачут между собой, но в целом варьируются между собой +/- одинаково, потому зависимости здесь нет. Разница между 2 св. таблицами тоже не велика. Это связано с тем, что эти значения являются случайными величинами и здесь больше подходит расчет коэф. корреляции*

- #### Как разные цели кредита влияют на его возврат в срок?

In [43]:
purpose_pivot = df_edited.pivot_table(index = 'purpose_after_lemmatization', values = 'debt', aggfunc = ['count', 'sum'], margins = True)
purpose_pivot.columns = ['Общее кол-во клиентов', 'Кол-во клиентов с долгами по кредиту']
purpose_pivot['ratio'] = purpose_pivot['Кол-во клиентов с долгами по кредиту'] / purpose_pivot['Общее кол-во клиентов']
purpose_pivot['ratio'] = purpose_pivot['ratio'].apply('{:.2%}'.format)
purpose_pivot

,Общее кол-во клиентов,Кол-во клиентов с долгами по кредиту,ratio
purpose_after_lemmatization,,,
автокредит,4306,403,9.36%
жилая недвижимость,8893,648,7.29%
коммерческая недвижимость,1311,99,7.55%
образование,4013,370,9.22%
ремонт жилья,607,35,5.77%
свадьба,2324,186,8.00%
All,21454,1741,8.12%


### Вывод

*Общая картина показывает, что цели кредита не влияют на его своевременное погошение, так как на кол-во клиентов, у которых имеются долги по погашению, приходит малая доля. Единственное, что можно заметить - клиенты, которые берут кредит на ремонт жилья, погашают кредит более успешно, чем другие другие*

### Шаг 4. Общий вывод

## **Общий вывод по работе**

### Изучение датафрейма
1. Первым шагом было изучение данных. В ходе проверки было выявлено: 2174 строк с пропущенными значениями в столбцах "Стаж" и "Доход", из 2174 строк 71 являлись дубликатами. Нашлись отрицательные значения в столбцах "Стаж" и "Количество детей", также были 0-ые значения в столбце "Возраст клиента". Столбец "Степень образования" имел не общий формат заполнения (Текст был записан либо прописью, либо заглавными). Столбец "Пол клиента" содержал данные "XNA", я так и не понял что это за ошибка, или это гендерное предрасположенность. Столбец "Цель кредита" содержал разные по содержанию текст, но имеющий общий смысловой вид.

*Причинами появления пропущенных и отрицательных данных могут быть различными (Технические (при перемещение данных из одной среды в другую), ошибка заполняюшего эти данные, ошибка ответственных по вводу данных в базу, ошибка людей, кто формировал данный отчет и т.д.)*

### Предобработка данных
1. Исправляем отрицательные значения в столбцах "Стаж" и "Количество детей" с помощью функцией abs(), которая возвадила в модуль отрицательные значения;

2. Заполняем пропуски в столбцах "Стаж" и "Доход". Столбец "Доход" заполняет средними величинами в зависимости от типа занятости. Столбец "Стаж" заполняем средними величинаи в зависимости от возраста клиента. 0-ые значения в столбце "Возраст клиента" заменяем средними величинами в зависимости от типа занятости

3. Убираем дубликаты с помощью метода drop_duplicates()

4. Замена типа данных было в столбцах "Стаж" и "Доход". Стаж считался в днях, а доход имел много цифр после целого значения (после точки/запятой), поэтому желательно иметь данные в целочичленном формате, а не в дробном. Использовался метод astype() со значением 'int'. to_numeric() отпадает, так как его конечный результат это вещественные числа, а не целые.

5. Чтобы провести категоризацию по целям кредита, следует вначале сделать лемматизацию столбца "Цель кредита", а после вытаскивать основное слово. Цель кредита запишем в новом столбце "main_purpose", предварительно заменив "жилье" на "недвижимость". Категоризация по уровню дохода сделаем с помощью метода qcut(), который позволит разбить по квантилям данных

### Работа с поставленными вопросами
1. Есть ли зависимость между наличием детей и возвратом кредита в срок?
*Исправляем значение 20 на 2 в столбце "Кол-во детей". Формируем сводную таблицу, в которой указываем общее число клиентов с детьми и клиентов, у которых есть дети и долг по кредиту. Находим % содержание и выявляем, что зависимости нет, так как на клиентов с детьми в независимости от числа детей и долгами по кредиту приходит малая доля*

2. Есть ли зависимость между семейным положением и возвратом кредита в срок?
*Соотношения между клиентами с долгами по кредиту и с клиентами без долгов выдают малые значения, для того, чтобы подвердить закономерность семейного положения и возврата кредита в срок. Хотя если рассматривать категорию "Не женат / не замужем", то можно сказать, что каждый десятый имеет долг по погашению, но даже так это не будет являться большой долей для подтверждения зависимости. Поэтому закономерности нет*

3. Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
*Общая картина показывает, что зависимость между уровнем дохода и возвратом кредита скачут между собой, но в целом варьируются между собой +/- одинаково, потому зависимости здесь нет. Было построены 2 св. таблицы для анализа, но они выдали примерно одинаковые результаты. Это связано с тем, что эти значения являются случайными величинами и здесь больше подходит расчет коэф. корреляции*

4. Как разные цели кредита влияют на его возврат в срок?
*Общая картина показывает, что цели кредита не влияют на его своевременное погошение, так как на кол-во клиентов, у которых имеются долги по погашению, приходит малая доля*